In [162]:
import pandas as pd
full_input = pd.read_clipboard()

In [118]:
import numpy as np

In [269]:
full_input.to_records()

rec.array([( 0,  0, 0.05, 2, 5, 13, 20, -1, -1, 0., 0., 0., 0., 0., 0., 0.005),
           ( 1,  1, 0.05, 2, 5, 13, 20, -1, -1, 0., 0., 0., 0., 0., 0., 0.005),
           ( 2,  2, 0.05, 2, 5, 13, 20, -1, -1, 0., 0., 0., 0., 0., 0., 0.005),
           ( 3,  3, 0.05, 2, 5, 13, 20, -1, -1, 0., 0., 0., 0., 0., 0., 0.005),
           ( 4,  4, 0.05, 2, 5, 13, 20, -1, -1, 0., 0., 0., 0., 0., 0., 0.005),
           ( 5,  5, 0.05, 3, 6, 14, 21, -1, -1, 0., 0., 0., 0., 0., 0., 0.005),
           ( 6,  6, 0.05, 3, 6, 14, 21, -1, -1, 0., 0., 0., 0., 0., 0., 0.005),
           ( 7,  7, 0.05, 3, 6, 14, 21, -1, -1, 0., 0., 0., 0., 0., 0., 0.005),
           ( 8,  8, 0.05, 3, 6, 14, 21, -1, -1, 0., 0., 0., 0., 0., 0., 0.005),
           ( 9,  9, 0.05, 3, 6, 14, 21, -1, -1, 0., 0., 0., 0., 0., 0., 0.005),
           (10, 10, 0.05, 2, 5, 13, 20, -1, -1, 0., 0., 0., 0., 0., 0., 0.005),
           (11, 11, 0.05, 2, 5, 13, 20, -1, -1, 0., 0., 0., 0., 0., 0., 0.005),
           (12, 12, 0.05, 2, 5, 13, 20, 

In [163]:
promo = full_input.loc[:,"Promo1":"Sink2"].to_numpy()
fed = full_input.loc[:,"FED"].to_numpy()
other = full_input.loc[:,"Other"].to_numpy()
spread = full_input.loc[:,"Promo1_spread":"Sink2_spread"].to_numpy()

In [151]:
other 

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0.])

In [145]:
# Custom formatting function
def custom_format(x):
    if x == int(x):
        return f"{x:.0f}"
    else:
        return f"{x:.2f}"

In [165]:
N = 21
def interest_rate_fed_forecast(N, promo, fed, other):
    # Initialize ir_forecast_rate with 'other' values
    ir_forecast_rate = np.tile(other[:, np.newaxis], (1, N))

    # Create row indices corresponding to each element in promo
    row_indices = np.arange(promo.shape[0])[:, np.newaxis]

    # Flatten row_indices to use with the flattened promo array
    row_indices_flat = np.repeat(row_indices, promo.shape[1])

    # Flatten promo and filter out invalid indices
    promo_flat = promo.flatten() - 1 
    valid_indices_flat = (promo_flat >= 0) & (promo_flat < N)

    # Set the values in ir_forecast_rate using advanced indexing
    ir_forecast_rate[row_indices_flat[valid_indices_flat], promo_flat[valid_indices_flat]] = fed[row_indices_flat[valid_indices_flat]]
    
    return ir_forecast_rate
    
def spread_forecast(N, promo, spread):
    spread_forecast = np.zeros((promo.shape[0], N))
    # Create row indices corresponding to each element in promo
    row_indices = np.arange(promo.shape[0])[:, np.newaxis]

    # Flatten row_indices to use with the flattened promo and spread arrays
    row_indices_flat = np.repeat(row_indices, promo.shape[1])

    # Flatten promo and spread, and filter out invalid indices in promo
    promo_flat = promo.flatten() - 1 
    spread_flat = spread.flatten()
    valid_indices_flat = (promo_flat >= 0) & (promo_flat < N)

    # Set the values in spread_forecast using advanced indexing
    spread_forecast[row_indices_flat[valid_indices_flat], promo_flat[valid_indices_flat]] = spread_flat[valid_indices_flat]
    
    return spread_forecast

def interest_rate_forecast(N,  promo, fed, other, spread):
    fed_forecast_ = interest_rate_fed_forecast(N, promo, fed, other)
    spread_forecast_ = spread_forecast(N, promo, spread)
    interest_rate_forecast_ = fed_forecast_ + spread_forecast_
    return interest_rate_forecast_

 
interest_forecast = interest_rate_forecast(N = 6, promo = promo,  fed = fed, other  = other, spread = spread)
interest_forecast[:10]
# interest_forecast
# spread.flatten()

array([[0.005, 0.05 , 0.005, 0.005, 0.05 , 0.005],
       [0.005, 0.05 , 0.005, 0.005, 0.05 , 0.005],
       [0.005, 0.05 , 0.005, 0.005, 0.05 , 0.005],
       [0.005, 0.05 , 0.005, 0.005, 0.05 , 0.005],
       [0.005, 0.05 , 0.005, 0.005, 0.05 , 0.005],
       [0.005, 0.005, 0.05 , 0.005, 0.005, 0.05 ],
       [0.005, 0.005, 0.05 , 0.005, 0.005, 0.05 ],
       [0.005, 0.005, 0.05 , 0.005, 0.005, 0.05 ],
       [0.005, 0.005, 0.05 , 0.005, 0.005, 0.05 ],
       [0.005, 0.005, 0.05 , 0.005, 0.005, 0.05 ]])

In [157]:
N = 6 
fed_forecast_ = interest_rate_fed_forecast(N, promo, fed, other)
spread_forecast_ = spread_forecast(N, promo, spread)
interest_rate_forecast_ = fed_forecast_ + spread_forecast_
# spread_forecast_[:10]
# fed_forecast_[:10]

In [96]:
monthly_forecast = pd.read_clipboard()

In [273]:
monthly_forecast.to_records()

rec.array([( 0,    0,    0,    0,    0,    0,     0),
           ( 1, 8500, 8500, 8500, 8500, 8500, 15700),
           ( 2, 7225, 8500, 8500, 8500, 8500, 15520),
           ( 3, 6141, 7225, 8500, 8500, 8500, 15187),
           ( 4, 5220, 7225, 7225, 8500, 8500, 14877),
           ( 5, 4437, 6141, 7225, 7225, 8500, 14433),
           ( 6, 3771, 6141, 7225, 7225, 7225, 14159),
           ( 7, 3206, 5220, 6141, 7225, 7225, 19326),
           ( 8, 2725, 5220, 6141, 7225, 7225, 19083),
           ( 9, 2316, 4437, 6141, 6141, 7225, 18439),
           (10, 1969, 4437, 5220, 6141, 7225, 17959),
           (11, 1673, 3771, 5220, 6141, 6141, 17240),
           (12, 1422, 3771, 5220, 6141, 6141, 16939),
           (13, 1209, 3206, 4437, 5220, 6141, 21599),
           (14, 1028, 3206, 4437, 5220, 6141, 21339),
           (15,  874, 2725, 4437, 5220, 6141, 20624),
           (16,  743, 2725, 3771, 5220, 5220, 19916),
           (17,  631, 2316, 3771, 4437, 5220, 19035),
           (18,  536, 2316, 

In [126]:
# monthly_forecast.shape, interest_forecast.shape
# monthly_forecast.to_numpy()[:interest_forecast.shape[0]]

In [129]:
monthly_forecast_np = monthly_forecast.to_numpy()[:interest_forecast.shape[0]].astype(float) 


In [166]:
pd.DataFrame(interest_forecast).head()

,0,1,2,3,4,5
0,0.005,0.05,0.005,0.005,0.05,0.005
1,0.005,0.05,0.005,0.005,0.05,0.005
2,0.005,0.05,0.005,0.005,0.05,0.005
3,0.005,0.05,0.005,0.005,0.05,0.005
4,0.005,0.05,0.005,0.005,0.05,0.005


In [168]:
monthly_interest_exp = (monthly_forecast_np * interest_forecast)/12

In [170]:
pd.DataFrame(monthly_interest_exp).applymap(custom_format).head()

,0,1,2,3,4,5
0,0,0,0,0,0,0
1,3.54,35.42,3.54,3.54,35.42,6.54
2,3.01,35.42,3.54,3.54,35.42,6.47
3,2.56,30.10,3.54,3.54,35.42,6.33
4,2.18,30.10,3.01,3.54,35.42,6.20


In [176]:
def rolling_sum(arr, min_periods=1):
    """
    Apply a rolling sum to each column of a 2D NumPy array.
    The window size for each column is determined by its index + 1.

    :param arr: A 2D NumPy array.
    :param min_periods: Minimum number of observations in window required to have a value.
    :return: A 2D NumPy array with rolling sums.
    """
    nrows, ncols = arr.shape
    result = np.empty_like(arr, dtype=float)

    # Iterate over each column
    for col in range(ncols):
        window_size = col + 1
        # Apply rolling sum
        for row in range(nrows):
            if row < window_size - min_periods:
                result[row, col] = np.nan  # Not enough data to fill window
            else:
                start = max(0, row - window_size + 1)
                result[row, col] = arr[start:row + 1, col].sum()

    return result

pd.DataFrame(rolling_sum(monthly_interest_exp,monthly_interest_exp.shape[0]))

,0,1,2,3,4,5
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,3.541667,35.416667,3.541667,3.541667,35.416667,6.541667
2,3.010417,70.833333,7.083333,7.083333,70.833333,13.008333
3,2.558750,65.520833,10.625000,10.625000,106.250000,19.336250
4,2.175000,60.208333,10.093750,14.166667,141.666667,25.535000
5,1.848750,32.662917,36.656250,13.635417,145.208333,85.672500
6,1.571250,5.117500,63.218750,13.104167,112.802083,144.668333
7,1.335833,4.733750,85.795833,12.572917,80.395833,218.651667
8,1.135417,4.350000,81.279167,12.041667,47.989583,291.697500
9,0.965000,4.023750,76.762500,11.590000,15.583333,362.198750


In [236]:
import pandas as pd
import numpy as np

def create_fake_loan_data(N):
    """
    Create a DataFrame with N fake loan data entries.

    Parameters:
    N (int): Number of loans to generate.

    Returns:
    pd.DataFrame: DataFrame containing the fake loan data.
    """
    # ID - incremental integer
    ids = np.arange(1, N + 1)

    # DT - fixed date 10/31/2023
    dates = ["10/31/2023"] * N

    # GL_BAL_AMT - random float between 10000 and 200000
    balances = np.round(np.random.uniform(10000, 200000, N),1)

    # TERM_CAPPED - random term between 1 and 6
    terms = np.random.randint(1, 7, N)

    # TIME_TO_MATURITY - random number not greater than TERM_CAPPED
    time_to_maturity = np.random.randint(1, terms + 1)

    # INT_RATE - random rate between 0.01 and 0.05
    interest_rates = np.random.uniform(0.01, 0.05, N)

    # Creating the DataFrame
    loan_data = pd.DataFrame({
        'ID': ids,
        'DT': dates,
        'GL_BAL_AMT': balances,
        'TERM_CAPPED': terms,
        'TIME_TO_MATURITY': time_to_maturity,
        'INT_RATE': interest_rates
    })
    loan_data['DT'] = pd.to_datetime(loan_data['DT'])
    loan_data['MATURITY_DT'] = loan_data.apply(lambda row: row['DT'] + pd.DateOffset(months=1)*row['TIME_TO_MATURITY'], axis=1)
    loan_data['ORIG_DT'] = loan_data.apply(lambda row: row['MATURITY_DT'] - pd.DateOffset(months=1)* row['TERM_CAPPED'], axis=1)

    return loan_data

# Example usage
loans = 100
loan_data = create_fake_loan_data(loans)
loan_data.head()  # Display the first few rows of the generated data
 

,ID,DT,GL_BAL_AMT,TERM_CAPPED,TIME_TO_MATURITY,INT_RATE,MATURITY_DT,ORIG_DT
0,1,2023-10-31,117636.4,1,1,0.037884,2023-11-30,2023-10-30
1,2,2023-10-31,180992.6,1,1,0.022159,2023-11-30,2023-10-30
2,3,2023-10-31,143364.1,1,1,0.016362,2023-11-30,2023-10-30
3,4,2023-10-31,71114.1,2,2,0.031093,2023-12-30,2023-10-30
4,5,2023-10-31,179066.6,4,4,0.038895,2024-02-29,2023-10-29


In [237]:
loan_data['INT_EXP_M'] =  loan_data['INT_RATE'] * loan_data['GL_BAL_AMT']

In [238]:
pivot_table = loan_data.pivot_table(index='TIME_TO_MATURITY', 
                                     columns='TERM_CAPPED', 
                                     values='GL_BAL_AMT', 
                                     aggfunc='sum')
pivot_table = pivot_table.sort_index(ascending=False).fillna(0.)

In [239]:
pivot_table

TERM_CAPPED,1,2,3,4,5,6
TIME_TO_MATURITY,,,,,,
6,0.0,0.0,0.0,0.0,0.0,102787.0
5,0.0,0.0,0.0,0.0,99948.5,175516.7
4,0.0,0.0,0.0,608383.6,173953.9,245720.7
3,0.0,0.0,619244.9,638178.6,248256.7,142339.8
2,0.0,981836.9,932272.7,647722.6,714358.6,382363.9
1,1612315.8,1223015.1,256209.4,448970.0,203516.9,218125.1


In [240]:
loan_data[loan_data['TERM_CAPPED']==4].TIME_TO_MATURITY.unique()

array([4, 1, 2, 3])

In [244]:
# Create an array of shape (N, 6) with zeros (or another value of your choice)
new_rows = np.zeros((N, 6))

# Concatenate the original array with the new rows
extended_array = np.concatenate((pivot_table.to_numpy(), new_rows), axis=0)

In [250]:
pd.DataFrame(extended_array).head(20).to_csv('test.csv')

In [258]:
extended_array_roll = rolling_sum(extended_array,extended_array.shape[1])
pd.DataFrame(extended_array_roll [pivot_table.shape[0]:]).applymap(custom_format)

,0,1,2,3,4,5
0,0,1223015.10,1188482.10,1734871.20,1340086.10,1164066.20
1,0,0,256209.40,1096692.60,1166132.20,988549.50
2,0,0,0,448970,917875.50,742828.80
3,0,0,0,0,203516.90,600489
4,0,0,0,0,0,218125.10
5,0,0,0,0,0,0
6,0,0,0,0,0,0
7,0,0,0,0,0,0
8,0,0,0,0,0,0
9,0,0,0,0,0,0


In [261]:
bal_existing = pd.read_clipboard()

In [267]:
bal_existing.to_dict()

{'1': {0: 0.4, 1: 0.4, 2: 0.4, 3: 0.4, 4: 0.4, 5: 0.4},
 '2': {0: 41.7, 1: 41.7, 2: 41.7, 3: 41.7, 4: 41.7, 5: 41.7},
 '3': {0: 0.4, 1: 0.4, 2: 0.4, 3: 0.4, 4: 0.4, 5: 0.4},
 '4': {0: 41.7, 1: 41.7, 2: 41.7, 3: 41.7, 4: 41.7, 5: 41.7},
 '5': {0: 0.4, 1: 0.4, 2: 0.4, 3: 0.4, 4: 0.4, 5: 0.4},
 '6': {0: 0.4, 1: 0.4, 2: 0.4, 3: 0.4, 4: 0.4, 5: 0.4}}

In [263]:
extended_array = np.concatenate((bal_existing.to_numpy(), new_rows), axis=0)

In [266]:
 
extended_array_roll = rolling_sum(extended_array,extended_array.shape[1])
pd.DataFrame(extended_array_roll [pivot_table.shape[0]-1:]).applymap(custom_format)

,0,1,2,3,4,5
0,0.40,83.40,1.20,166.80,2,2.40
1,0,41.70,0.80,125.10,1.60,2
2,0,0,0.40,83.40,1.20,1.60
3,0,0,0,41.70,0.80,1.20
4,0,0,0,0,0.40,0.80
5,0,0,0,0,0,0.40
6,0,0,0,0,0,0
7,0,0,0,0,0,0
8,0,0,0,0,0,0
9,0,0,0,0,0,0
